# Ingestion and Cleaning of the academy csv files
In the academy bucket, there is data stored in csv files. This data describes each candidates assessed scores across their time in the academy.

In [1]:
import os, json, csv, boto3, datetime
import pandas as pd

In [2]:
s3 = boto3.client('s3')

### We create a function to list all object within a bucket

In [3]:
def list_objects(bucket, prefix):
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    if 'Contents' in response:
        return [obj['Key'] for obj in response['Contents'] if obj['Key'] != prefix]
    return []

In [4]:
def list_all_objects(bucket, prefix):
    all_objects = []
    continuation_token = None
    
    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
        
        if 'Contents' in response:
            all_objects.extend([obj['Key'] for obj in response['Contents']])
        
        if not response.get('NextContinuationToken'):
            break
        
        continuation_token = response['NextContinuationToken']
    
    return all_objects

## We load in the academy data.
Since we know that the academy bucket only contains these csv files, we don't need to perform any checks.

In [5]:
def load_academy_data(bucket, prefix):
    files = list_all_objects(bucket, prefix)
    data_frames = []
    for file_key in files:
        obj = s3.get_object(Bucket=bucket, Key=file_key)
        df = pd.read_csv(obj['Body'])
        file_name = file_key.split("/")[1].split(".")[0]
        df.insert(0,'filename', file_name, True)
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [6]:
academy_data = load_academy_data('data-402-final-project', 'Academy/')

In [7]:
def clean_whitespace(text):
    try:
        return text.strip()
    except:
        return text

In [8]:
cols = list(academy_data.columns.values)
for col in cols:
    academy_data[col] = academy_data[col].apply(clean_whitespace)

### Next we need to extract the caterogry, stream name, and start date from the file name

In [9]:
def get_category(filename):
    splits = filename.split("_")
    category = splits[0]
    return category

In [10]:
academy_data['Category'] = academy_data['filename'].apply(get_category)

In [11]:
def get_stream(filename):
    splits = filename.split("_")
    stream_name = "".join(splits[0:2])
    return stream_name

In [12]:
academy_data['Stream'] = academy_data['filename'].apply(get_stream)

In [13]:
def get_date(filename):
    splits = filename.split("_")
    date = datetime.datetime.strptime(splits[2], "%Y-%m-%d").date()
    return date

In [14]:
academy_data['Date'] = academy_data['filename'].apply(get_date)

In [15]:
academy_data = academy_data[['Category', 'Stream', 'Date'] + [col for col in academy_data if col not in ['Category','Stream', 'Date','filename']]]

### Next we check for any duplicated values

In [16]:
academy_data.duplicated().sum()

0

There are no duplicate values in our table
### Finally, we look to see if where there are null values
We don't want any null values in Category, Stram, Date, name, and trainer.

In [17]:
academy_data.isnull().sum()

Category              0
Stream                0
Date                  0
name                  0
trainer               0
                   ... 
Independent_W10     235
Determined_W10      235
Professional_W10    235
Studious_W10        235
Imaginative_W10     235
Length: 65, dtype: int64

In [18]:
display(academy_data)

,Category,Stream,Date,name,trainer,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,Business,Business20,2019-02-11,Quintus Penella,Gregor Gomez,1,2,2,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Business,Business20,2019-02-11,Simon Murrey,Gregor Gomez,6,1,1,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Business,Business20,2019-02-11,Gustaf Lude,Gregor Gomez,6,4,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Business,Business20,2019-02-11,Yolanda Fosse,Gregor Gomez,2,1,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Business,Business20,2019-02-11,Lynnett Swin,Gregor Gomez,2,2,4,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Engineering,Engineering29,2019-12-30,Moritz Mosedall,Mohammad Velazquez,1,1,5,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,Engineering,Engineering29,2019-12-30,Chaim Inseal,Mohammad Velazquez,1,3,3,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
394,Engineering,Engineering29,2019-12-30,Gertruda Syddie,Mohammad Velazquez,3,1,2,8,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,Engineering,Engineering29,2019-12-30,Thom Derwin,Mohammad Velazquez,3,7,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
import pyodbc
server = '127.0.0.1' # Replace this with the actual name of your SQL Edge Docker container
username = 'sa' # SQL Server username
password = 'Ducks123' # Replace this with the actual SA password from your deployment
database = 'master' # Replace this with the actual database name from your deployment. If you do not have a database created, you can use Master database.
db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"
conn = pyodbc.connect(db_connection_string, autocommit=True)


In [41]:
import pandas as pd
import sqlalchemy as sa

connection_string = (
    'Driver=ODBC Driver 17 for SQL Server;'
    'Server=127.0.0.1;'
    'Database=FinalProject;'
    'UID=sa;'
    'PWD=password;'
    'Trusted_Connection=no;'
)
connection_url = sa.engine.URL.create(
    "mssql+pyodbc", 
    query=dict(odbc_connect=connection_string)
)
engine = sa.create_engine(connection_url, fast_executemany=True)

# # Deleting existing data in SQL Table:-
# with engine.begin() as conn:
#     conn.exec_driver_sql("DELETE FROM SchemaName.TableName")

# upload the DataFrame
academy_data.to_sql("TableName", engine, schema="dbo", if_exists="replace", index=False)

-1